# Day 5

In [1]:
import re

In [2]:
from ipynb.fs.defs.utils import read_lines 
puzzle_input = read_lines("day5.txt")

In [3]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""".splitlines()

In [4]:
def get_min_location_seed(almanac):
    seeds = map(int, almanac[0].split()[1:])
    mappings = get_mappings(almanac[2:])
    location_to_seed = {map_forward(seed, mappings): seed for seed in seeds}
    return min(location_to_seed) # location_to_seed[min(location_to_seed)]

In [5]:
def get_mappings(map_defs):
    mappings = []
    for l in map_defs:
        if 'map' in l:
            current_map = []
        elif len(l) > 0:
            to_start, from_start, length = map(int, l.split())
            current_map.append((to_start, from_start, length))
        elif len(current_map) > 0:
            mappings.append(current_map)
            current_map = {}
        else:
            print('unable to handle line:' + l)

    if len(current_map) > 0:
        mappings.append(current_map)
    return mappings

In [6]:
def map_forward(seed, mappings):
    value = seed
    for map_step in mappings:
        value = map_value(value, map_step)
    print(f'seed {seed} maps to location {value}')
    return value

In [7]:
def map_value(value, map_def):
    for to_start, from_start, length in map_def:
        if value >= from_start and value < from_start + length:
            return value - from_start + to_start
    return value

In [8]:
get_mappings(test_input[2:])

[[(50, 98, 2), (52, 50, 48)],
 [(0, 15, 37), (37, 52, 2), (39, 0, 15)],
 [(49, 53, 8), (0, 11, 42), (42, 0, 7), (57, 7, 4)],
 [(88, 18, 7), (18, 25, 70)],
 [(45, 77, 23), (81, 45, 19), (68, 64, 13)],
 [(0, 69, 1), (1, 0, 69)],
 [(60, 56, 37), (56, 93, 4)]]

In [9]:
map_forward(79, get_mappings(test_input[2:]))

seed 79 maps to location 82


82

In [10]:
map_value(79, [(50, 98, 2), (52, 50, 48)])

81

In [11]:
get_min_location_seed(puzzle_input)

seed 1778931867 maps to location 3527214727
seed 1436999653 maps to location 411334602
seed 3684516104 maps to location 1846863896
seed 2759374 maps to location 1011130144
seed 1192793053 maps to location 107430936
seed 358764985 maps to location 2144123292
seed 1698790056 maps to location 2030474631
seed 76369598 maps to location 2256093975
seed 3733854793 maps to location 905605726
seed 214008036 maps to location 3270099765
seed 4054174000 maps to location 633691295
seed 171202266 maps to location 235972814
seed 3630057255 maps to location 3827359985
seed 25954395 maps to location 2918856711
seed 798587440 maps to location 2586721517
seed 316327323 maps to location 3414553015
seed 290129780 maps to location 2926997052
seed 7039123 maps to location 1015409893
seed 3334326492 maps to location 3315197680
seed 246125391 maps to location 3101317175


107430936

In [12]:
def get_seed_ranges(seed_defs):
    seed_list = list(map(int, seed_defs.split()[1:]))
    return [seed_list[i:i + 2] for i in range(0, len(seed_list), 2)]

In [13]:
def get_overlap(range1, range2):
    s1, l1 = range1
    s2, l2 = range2
    overlap = (max(s1, s2), min(s1 + l1, s2 + l2) - max(s1, s2))
    if overlap[1] <= 0:
        return None
    return overlap

In [14]:
def map_range(in_range, map_defs):
    out_ranges = []
    processed_ranges = []
    for to_start, from_start, length in map_defs:
        overlap = get_overlap(in_range, (from_start, length))
        if overlap:
            out_ranges.append((overlap[0] - from_start + to_start, overlap[1]))
            processed_ranges.append(overlap)
    processed_ranges = sorted(processed_ranges,key=lambda x: x[0])
    c_value = in_range[0]
    for p_range in processed_ranges:
        if c_value < p_range[0]:
            out_ranges.append((c_value, p_range[0] - c_value))
            c_value = p_range[0] + p_range[1]
        elif c_value == p_range[0]:
            c_value += p_range[1]
        else:
            print(f'error overlapping mapping {in_range} using {map_def} to {out_ranges}. processed {processed_ranges}')
    if c_value < in_range[0] + in_range[1]:
        out_ranges.append((c_value, in_range[0] + in_range[1] - c_value))
    return out_ranges

In [15]:
def map_ranges(in_ranges, map_def):
    out_ranges = []
    for r in in_ranges:
        out_ranges += map_range(r, map_def)
    return out_ranges

In [16]:
def map_range_forward(seed_range, mappings):
    c_ranges = [seed_range]
    for map_def in mappings:
        c_ranges = map_ranges(c_ranges, map_def)
    return c_ranges

In [17]:
map_range_forward((55, 13), get_mappings(test_input[2:]))

[(86, 4), (94, 3), (56, 4), (97, 2)]

In [18]:
def get_min_location_seed_range(almanac):
    seed_ranges = get_seed_ranges(almanac[0])
    mappings = get_mappings(almanac[2:])
    min_location = None
    for seed_range in seed_ranges:
        mappend_ranges = map_range_forward(seed_range, mappings)
        min_range = min(mappend_ranges, key= lambda x: x[0])
        if min_location is None or min_range[0] < min_location:
            min_location = min_range[0]
    return min_location

In [19]:
map_range((79, 14), [(10, 60, 20), (120, 85, 3)])

[(29, 1), (120, 3), (80, 5), (88, 5)]

In [20]:
get_mappings(test_input[2:])

[[(50, 98, 2), (52, 50, 48)],
 [(0, 15, 37), (37, 52, 2), (39, 0, 15)],
 [(49, 53, 8), (0, 11, 42), (42, 0, 7), (57, 7, 4)],
 [(88, 18, 7), (18, 25, 70)],
 [(45, 77, 23), (81, 45, 19), (68, 64, 13)],
 [(0, 69, 1), (1, 0, 69)],
 [(60, 56, 37), (56, 93, 4)]]

In [21]:
get_min_location_seed_range(puzzle_input)

23738616

In [22]:
get_seed_ranges(test_input[0])

[[79, 14], [55, 13]]